In [ ]:
!pip install transformers smolagents --upgrade -q

In [ ]:
!pip install markdownify duckduckgo-search spaces gradio-tools langchain langchain-community langchain-huggingface faiss-cpu --upgrade -q

In [ ]:
!pip install markdownify duckduckgo-search spaces gradio-tools langchain langchain-community langchain-huggingface faiss-cpu --upgrade -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 6.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take 

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from smolagents import InferenceClientModel

model_id = "Qwen/Qwen2.5-72B-Instruct"
model = InferenceClientModel(model_id)

In [ ]:
from smolagents import CodeAgent, ToolCallingAgent,DuckDuckGoSearchTool, VisitWebpageTool

web_agent = ToolCallingAgent(
    tools=[DuckDuckGoSearchTool(), VisitWebpageTool()],
    model=model,
    name="search_agent",
    description="Runs web searches for you. Give it your query as an argument.",
)

In [ ]:
from smolagents import CodeAgent, Tool, load_tool
#from transformers import load_tool, CodeAgent

prompt_generator_tool = Tool.from_space("sergiopaniego/Promptist", name="generator_tool", description="Optimizes user input into model-preferred prompts")

Loaded as API: https://sergiopaniego-promptist.hf.space ✔


/usr/local/lib/python3.12/dist-packages/smolagents/tools.py:666: UserWarning: Since `api_name` was not defined, it was automatically set to the first available API: `/predict`.
  warnings.warn(


In [ ]:
image_generation_tool = load_tool("m-ric/text-to-image", trust_remote_code=True)
image_generation_agent = CodeAgent(tools=[prompt_generator_tool, image_generation_tool], model=model,name="image_generation_agent",
    description="Generates images from text prompts. Give it your prompt as an argument.",
)

In [ ]:
manager_agent = CodeAgent(
    tools=[],
    model=model,
    managed_agents=[web_agent, image_generation_agent],
    additional_authorized_imports=["time", "datetime", "PIL"],
)

In [ ]:
# manager_agent.run("How many years ago was Stripe founded?")

In [ ]:
result = manager_agent.run(
    "Improve this prompt, then generate an image of it. A rabbit wearing a space suit."
)
print(result)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Improve this prompt, then generate an image of it. A rabbit wearing a space suit.                               │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-72B-Instruct ──────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import re                                                                                                        
                                                                                                                   
  # Original prompt                                                                                                
  original_prompt = "A rabbit wearing a space suit."                                                               
                                                                                                                   
  # Improve the prompt                                                                                             
  improved_prompt = re.sub(r"A rabbit wearing a space suit.",                                                      
                           "A fluffy white rabbit wearing a sleek, modern space suit with a transparent helmet,    
  standing on a rocky Martian surface under a reddish sky.",                                                       
                           original_prompt)                                                                        
                                                                                                                   
  # Generate the image                                                                                             
  image = image_generation_agent(task="Generate an image based on the improved prompt.",                           
  additional_args={"prompt": improved_prompt})                                                                     
  print(image)                                                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

╭─────────────────────────────────────── New run - image_generation_agent ────────────────────────────────────────╮
│                                                                                                                 │
│ You're a helpful agent named 'image_generation_agent'.                                                          │
│ You have been submitted this task by your manager.                                                              │
│ ---                                                                                                             │
│ Task:                                                                                                           │
│ Generate an image based on the improved prompt.                                                                 │
│ ---                                                                                                             │
│ You're helping your manager solve a wider task: so make sure to not provide a one-line answer, but give as much │
│ information as possible to give them a clear understanding of the answer.                                       │
│                                                                                                                 │
│ Your final_answer WILL HAVE to contain these parts:                                                             │
│ ### 1. Task outcome (short version):                                                                            │
│ ### 2. Task outcome (extremely detailed version):                                                               │
│ ### 3. Additional context (if relevant):                                                                        │
│                                                                                                                 │
│ Put all these in your final_answer tool, everything that you do not pass as an argument to final_answer will be │
│ lost.                                                                                                           │
│ And even if your task resolution is not successful, please return as much context as possible, so that your     │
│ manager can act upon this feedback.                                                                             │
│ You have been provided with these additional arguments, that you can access directly using the keys as          │
│ variables:                                                                                                      │
│ {'prompt': 'A fluffy white rabbit wearing a sleek, modern space suit with a transparent helmet, standing on a   │
│ rocky Martian surface under a reddish sky.'}.                                                                   │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-72B-Instruct ──────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(image)                                                                                              
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: Here is the final answer from your managed agent 'image_generation_agent':
{'task outcome (short version)': 'An image has been generated based on the provided prompt.', 'task outcome 
(extremely detailed version)': "Utilizing the image_generator tool, an image has been created that depicts a fluffy
white rabbit wearing a sleek, modern space suit with a transparent helmet, standing on a rocky Martian surface 
under a reddish sky. The image captures the intricate details of the rabbit's fur, the sleek design of the space 
suit, and the rugged Martian terrain with the characteristic reddish sky.", 'additional context (if relevant)': 
'The prompt was clear and detailed, which helped in generating a high-quality and visually appealing image. If 
further customization or adjustments are needed, additional parameters or details can be added to the prompt to 
refine the image.'}

[Step 2: Duration 1.78 seconds| Input tokens: 5,093 | Output tokens: 186]

Here is the final answer from your managed agent 'image_generation_agent':
{'task outcome (short version)': 'An image has been generated based on the provided prompt.', 'task outcome (extremely detailed version)': "Utilizing the image_generator tool, an image has been created that depicts a fluffy white rabbit wearing a sleek, modern space suit with a transparent helmet, standing on a rocky Martian surface under a reddish sky. The image captures the intricate details of the rabbit's fur, the sleek design of the space suit, and the rugged Martian terrain with the characteristic reddish sky.", 'additional context (if relevant)': 'The prompt was clear and detailed, which helped in generating a high-quality and visually appealing image. If further customization or adjustments are needed, additional parameters or details can be added to the prompt to refine the image.'}


In [ ]:
# from IPython.display import Image, display
# display(Image(filename=result))

In [ ]:
#

In [78]:
manager_agent.visualize()

CodeAgent | Qwen/Qwen2.5-72B-Instruct
├── ✅ Authorized imports: ['time', 'datetime', 'PIL']
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name         ┃ Description                                   ┃ Arguments                                    ┃
│   ┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   │ final_answer │ Provides a final answer to the given problem. │ answer (`any`): The final answer to the      │
│   │              │                                               │ problem                                      │
│   └──────────────┴───────────────────────────────────────────────┴──────────────────────────────────────────────┘
└── 🤖 Managed agents:
    ├── search_agent | ToolCallingAgent | Qwen/Qwen2.5-72B-Instruct
    │   ├── 📝 Description: Runs web searches for you. Give it your query as an argument.
    │   └── 🛠️ Tools:
    │       ┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
    │       ┃ Name          ┃ Description                              ┃ Arguments                                ┃
    │       ┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
    │       │ web_search    │ Performs a duckduckgo web search based   │ query (`string`): The search query to    │
    │       │               │ on your query (think a Google search)    │ perform.                                 │
    │       │               │ then returns the top search results.     │                                          │
    │       │ visit_webpage │ Visits a webpage at the given url and    │ url (`string`): The url of the webpage   │
    │       │               │ reads its content as a markdown string.  │ to visit.                                │
    │       │               │ Use this to browse webpages.             │                                          │
    │       │ final_answer  │ Provides a final answer to the given     │ answer (`any`): The final answer to the  │
    │       │               │ problem.                                 │ problem                                  │
    │       └───────────────┴──────────────────────────────────────────┴──────────────────────────────────────────┘
    └── image_generation_agent | CodeAgent | Qwen/Qwen2.5-72B-Instruct
        ├── ✅ Authorized imports: []
        ├── 📝 Description: Generates images from text prompts. Give it your prompt as an argument.
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
            ┃ Name            ┃ Description                             ┃ Arguments                               ┃
            ┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
            │ generator_tool  │ Optimizes user input into               │ plain_text (`string`):                  │
            │                 │ model-preferred prompts                 │                                         │
            │ image_generator │ This tool creates an image according to │ prompt (`string`): The image generator  │
            │                 │ a prompt, which is a text description.  │ prompt. Don't hesitate to add details   │
            │                 │                                         │ in the prompt to make the image look    │
            │                 │                                         │ better, like 'high-res,                 │
            │                 │                                         │ photorealistic', etc.                   │
            │ final_answer    │ Provides a final answer to the given    │ answer (`any`): The final answer to the │
            │                 │ problem.                                │ problem                   

In [ ]:
!pip install streamlit -q

In [110]:
%%writefile dasara.py

import streamlit as st
from smolagents import CodeAgent, ToolCallingAgent, DuckDuckGoSearchTool, VisitWebpageTool, Tool, load_tool, InferenceClientModel

# Assuming manager_agent is already defined and configured in the previous cells
# If not, you would need to include the code to initialize it here.

st.title("SmolAgents Chatbot")

# Initialize chat history
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display chat messages from history on app rerun
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# React to user input
if prompt := st.chat_input("Enter your query"):
    # Display user message in chat message container
    st.chat_message("user").markdown(prompt)
    # Add user message to chat history
    st.session_state.messages.append({"role": "user", "content": prompt})

    with st.chat_message("assistant"):
        # Pass the query to the manager_agent and get the response
        response = manager_agent.run(prompt)
        st.markdown(response)
    # Add assistant response to chat history
    st.session_state.messages.append({"role": "assistant", "content": response})

Overwriting dasara.py


To run this Streamlit app, save the code as a Python file (e.g., `app.py`) and run the following command in your terminal:

In [111]:
# !streamlit run dasara.py --server.port=80


Alternatively, you can use `ngrok` to expose your local Streamlit app to the internet:

In [101]:
# !pip install pyngrok

In [98]:
# from pyngrok import ngrok
# from google.colab import userdata

# # Kill any previous tunnels
# ngrok.kill()

# # Get the authtoken from Colab secrets
# NGROK_AUTH_TOKEN = userdata.get("NGROK_AUTH_TOKEN")

# # Connect to ngrok with the authtoken
# ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# # Launch a public URL for port 8501
# public_url = ngrok.connect(8501)
# print("🔗 Your Streamlit app is live at:", public_url)

# # Run Streamlit app
# !streamlit run dasara.py --server.port 8501

In [112]:
! wget -q -O - ipv4.icanhazip.com


34.9.82.58


In [115]:
!streamlit run dasara.py &>/content/logs.txt &


In [116]:
!npx localtunnel --port 8501


⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧your url is: https://olive-jeans-agree.loca.lt
^C
